In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import tensorflow as tf
import tensorflow.keras.backend as K
import math
from scipy.stats import skew, kurtosis
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Dropout, Conv1D, MaxPooling1D, Flatten, TimeDistributed, Activation, Reshape

#Load and Prepare Data

In [ ]:
# load the data for each subset
def load_dataframe(subset_number):
    directory = 'C-MAPSS/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    train_url = f"https://raw.githubusercontent.com/DemetraAS/C-MAPSS/main/train_FD00{subset_number}.txt"
    os.system(f"wget -q {train_url} -O {directory}train_FD00{subset_number}.txt")
    turbo_train = pd.read_csv(f"{directory}train_FD00{subset_number}.txt", sep='\s+', header=None, index_col=False)

    test_url = f"https://raw.githubusercontent.com/DemetraAS/C-MAPSS/main/test_FD00{subset_number}.txt"
    os.system(f"wget -q {test_url} -O {directory}test_FD00{subset_number}.txt")
    turbo_test = pd.read_csv(f"{directory}test_FD00{subset_number}.txt", sep='\s+', header=None, index_col=False)

    rul_url = f"https://raw.githubusercontent.com/DemetraAS/C-MAPSS/main/RUL_FD00{subset_number}.txt"
    os.system(f"wget -q {rul_url} -O {directory}RUL_FD00{subset_number}.txt")
    y_RUL = pd.read_csv(f"{directory}RUL_FD00{subset_number}.txt", sep='\s+', header=None, index_col=False)

    return turbo_train, turbo_test, y_RUL

In [ ]:
turbo_train_1, turbo_test_1, y_RUL_1 = load_dataframe(1)
turbo_train_2, turbo_test_2, y_RUL_2 = load_dataframe(2)
turbo_train_3, turbo_test_3, y_RUL_3 = load_dataframe(3)
turbo_train_4, turbo_test_4, y_RUL_4 = load_dataframe(4)

In [ ]:
turbo_train_1

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640


In [ ]:
def count_engines(turbofan):
  unique_engines = turbofan[0].unique()
  num_engines = len(unique_engines)
  return num_engines

In [ ]:
nr_train_1=count_engines(turbo_train_1)
nr_train_2=count_engines(turbo_train_2)
nr_train_3=count_engines(turbo_train_3)
nr_train_4=count_engines(turbo_train_4)
nr_test_1=count_engines(turbo_test_1)
nr_test_2=count_engines(turbo_test_2)
nr_test_3=count_engines(turbo_test_3)
nr_test_4=count_engines(turbo_test_4)

In [ ]:
# assign the original RUL to the training set (here RUL decreases linearly)
def original_RUL_train(turbo_train):
  turbo_train.sort_values(by=[0, 1], inplace=True)
  turbo_train['RUL'] = turbo_train.groupby(0).cumcount(ascending=False)
  return turbo_train

In [ ]:
turbo_train_1=original_RUL_train(turbo_train_1)
turbo_train_2=original_RUL_train(turbo_train_2)
turbo_train_3=original_RUL_train(turbo_train_3)
turbo_train_4=original_RUL_train(turbo_train_4)

In [ ]:
turbo_train_1

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735,4
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594,3
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333,2
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640,1


In [ ]:
 # assign the original RUL to the testing set (RUL decreases linearly)
def original_RUL_test(turbo_test, y_RUL):
  y_RUL_series=y_RUL[0]
  turbo_test.sort_values(by=[0, 1], inplace=True)
  adjusted_engine_numbers = turbo_test[0].values - 1
  turbo_test['RUL'] = turbo_test.groupby(0).cumcount(ascending=False) + y_RUL_series.iloc[adjusted_engine_numbers].values
  return turbo_test

In [ ]:
turbo_test_1=original_RUL_test(turbo_test_1, y_RUL_1)
turbo_test_2=original_RUL_test(turbo_test_2, y_RUL_2)
turbo_test_3=original_RUL_test(turbo_test_3, y_RUL_3)
turbo_test_4=original_RUL_test(turbo_test_4, y_RUL_4)

In [ ]:
turbo_test_1

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,RUL
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,142
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,141
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,140
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,139
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974,24
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771,23
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051,22
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699,21


In [ ]:
#concatenate training and testing set for subsequent operations (reset engines indexes for training set)
def complete_dataframe(turbo_train, turbo_test):
  turbo_test_reset = turbo_test.reset_index(drop=True)
  last_engine = turbo_train[0].iloc[-1]
  turbo_test_reset[0] += last_engine
  turbofan = pd.concat([turbo_train, turbo_test_reset], ignore_index=True)
  return turbofan

In [ ]:
turbofan1=complete_dataframe(turbo_train_1, turbo_test_1)
turbofan2=complete_dataframe(turbo_train_2, turbo_test_2)
turbofan3=complete_dataframe(turbo_train_3, turbo_test_3)
turbofan4=complete_dataframe(turbo_train_4, turbo_test_4)

In [ ]:
turbofan1

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33722,200,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974,24
33723,200,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771,23
33724,200,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051,22
33725,200,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699,21


In [ ]:
# rename columns as in the original paper
def rename_columns(turbofan):
  new_columns = ['ENr', 'C', 'OP-1', 'OP-2', 'OP-3',
                 'T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi',
                 'NRf', 'NRc', 'BPR', 'farB', 'htBleed', 'Nf-dmd', 'PCNfR-dmd', 'W31', 'W32', 'RUL']
  turbofan.columns = new_columns
  return turbofan

In [ ]:
turbofan1=rename_columns(turbofan1)
turbofan2=rename_columns(turbofan2)
turbofan3=rename_columns(turbofan3)
turbofan4=rename_columns(turbofan4)

In [ ]:
turbofan1

,ENr,C,OP-1,OP-2,OP-3,T2,T24,T30,T50,P2,...,NRf,NRc,BPR,farB,htBleed,Nf-dmd,PCNfR-dmd,W31,W32,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33722,200,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974,24
33723,200,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771,23
33724,200,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051,22
33725,200,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699,21


#Preprocessing

##Normalization

In [ ]:
# perform z-score normalization (standardization)
def z_score(turbofan):
  scaler = StandardScaler()
  normalized_turbo = turbofan.copy()
  columns_to_normalize = turbofan.columns.drop(['ENr', 'C', 'RUL'])
  normalized_turbo[columns_to_normalize] = scaler.fit_transform(turbofan[columns_to_normalize])
  return normalized_turbo

In [ ]:
normalized_turbo1=z_score(turbofan1)
normalized_turbo2=z_score(turbofan2)
normalized_turbo3=z_score(turbofan3)
normalized_turbo4=z_score(turbofan4)

In [ ]:
normalized_turbo1

,ENr,C,OP-1,OP-2,OP-3,T2,T24,T30,T50,P2,...,NRf,NRc,BPR,farB,htBleed,Nf-dmd,PCNfR-dmd,W31,W32,RUL
0,1,1,-0.314711,-1.373690,0.0,0.0,-1.644975,0.020217,-0.794964,-3.552714e-15,...,-0.981906,-0.199424,-0.461431,-1.387779e-17,-0.655669,0.0,0.0,1.254019,1.091572,191
1,1,2,0.870755,-1.032895,0.0,0.0,-0.949920,0.383216,-0.493750,-3.552714e-15,...,-0.242468,-0.634651,-0.113571,-1.387779e-17,-0.655669,0.0,0.0,0.902637,1.136638,190
2,1,3,-1.956125,1.011877,0.0,0.0,-0.528674,-0.272579,-0.368046,-3.552714e-15,...,-0.834018,-0.528438,-0.509509,-1.387779e-17,-2.018065,0.0,0.0,0.609819,0.358754,189
3,1,4,0.323617,-0.010509,0.0,0.0,-0.528674,-1.162954,-0.644357,-3.552714e-15,...,-0.094581,-0.491813,-1.912263,-1.387779e-17,-0.655669,0.0,0.0,0.199874,0.649726,188
4,1,5,-0.861849,-0.692099,0.0,0.0,-0.486549,-1.152680,-0.128498,-3.552714e-15,...,-0.686131,-0.493645,-0.181446,-1.387779e-17,0.025530,0.0,0.0,0.317001,0.948535,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33722,200,194,2.238601,-0.010509,0.0,0.0,1.345871,1.689670,1.006393,-3.552714e-15,...,-1.277681,4.357951,1.009197,-1.387779e-17,0.706728,0.0,0.0,-1.147090,-1.079451,24
33723,200,195,-0.497090,-0.351304,0.0,0.0,1.303746,1.045861,1.748756,-3.552714e-15,...,0.053306,4.209620,0.435087,-1.387779e-17,1.387926,0.0,0.0,-1.615598,-0.298627,23
33724,200,196,-0.269116,-1.032895,0.0,0.0,1.767116,0.610947,-0.057345,-3.552714e-15,...,-0.686131,4.599676,0.596290,-1.387779e-17,1.387926,0.0,0.0,-1.322780,-1.004014,22
33725,200,197,-1.728151,0.330287,0.0,0.0,1.387995,0.926002,1.429753,-3.552714e-15,...,-0.094581,4.797451,0.997885,-1.387779e-17,1.387926,0.0,0.0,-1.088526,-0.369166,21


In [ ]:
#split dataframe into train and test
def split_dataframe(normalized_turbofan, nr_train):
  turbo_RUL_train = normalized_turbofan[(normalized_turbofan['ENr'] <= nr_train)]
  turbo_RUL_test = normalized_turbofan[(normalized_turbofan['ENr'] > nr_train)]
  return turbo_RUL_train, turbo_RUL_test

In [ ]:
turbo1_train, turbo1_test=split_dataframe(normalized_turbo1, nr_train_1)
turbo2_train, turbo2_test=split_dataframe(normalized_turbo2, nr_train_2)
turbo3_train, turbo3_test=split_dataframe(normalized_turbo3, nr_train_3)
turbo4_train, turbo4_test=split_dataframe(normalized_turbo4, nr_train_4)

In [ ]:
turbo1_train

,ENr,C,OP-1,OP-2,OP-3,T2,T24,T30,T50,P2,...,NRf,NRc,BPR,farB,htBleed,Nf-dmd,PCNfR-dmd,W31,W32,RUL
0,1,1,-0.314711,-1.373690,0.0,0.0,-1.644975,0.020217,-0.794964,-3.552714e-15,...,-0.981906,-0.199424,-0.461431,-1.387779e-17,-0.655669,0.0,0.0,1.254019,1.091572,191
1,1,2,0.870755,-1.032895,0.0,0.0,-0.949920,0.383216,-0.493750,-3.552714e-15,...,-0.242468,-0.634651,-0.113571,-1.387779e-17,-0.655669,0.0,0.0,0.902637,1.136638,190
2,1,3,-1.956125,1.011877,0.0,0.0,-0.528674,-0.272579,-0.368046,-3.552714e-15,...,-0.834018,-0.528438,-0.509509,-1.387779e-17,-2.018065,0.0,0.0,0.609819,0.358754,189
3,1,4,0.323617,-0.010509,0.0,0.0,-0.528674,-1.162954,-0.644357,-3.552714e-15,...,-0.094581,-0.491813,-1.912263,-1.387779e-17,-0.655669,0.0,0.0,0.199874,0.649726,188
4,1,5,-0.861849,-0.692099,0.0,0.0,-0.486549,-1.152680,-0.128498,-3.552714e-15,...,-0.686131,-0.493645,-0.181446,-1.387779e-17,0.025530,0.0,0.0,0.317001,0.948535,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.177926,-1.032895,0.0,0.0,1.872428,1.437968,2.529067,-3.552714e-15,...,2.567393,-0.261686,1.690777,-1.387779e-17,2.750323,0.0,0.0,-2.084107,-3.273007,4
20627,100,197,-0.725065,-1.714485,0.0,0.0,1.977739,2.554361,3.116080,-3.552714e-15,...,1.975843,-0.328832,2.208325,-1.387779e-17,1.387926,0.0,0.0,-3.196816,-1.451738,3
20628,100,198,0.186832,-0.010509,0.0,0.0,1.724992,2.205060,2.475703,-3.552714e-15,...,2.271618,-0.051092,3.642188,-1.387779e-17,3.431521,0.0,0.0,-2.376925,-3.666848,2
20629,100,199,-0.497090,1.011877,0.0,0.0,1.324808,2.684493,2.280032,-3.552714e-15,...,2.123731,-0.158526,2.915358,-1.387779e-17,1.387926,0.0,0.0,-3.255380,-2.386375,1


##Prepare for Training

In [ ]:
#split data into arrays
def train_split(turbo_train, turbo_test):
  X_train=turbo_train.iloc[:, :-1]
  y_train=turbo_train.iloc[:, -1]

  X_test=turbo_test.iloc[:, :-1]
  y_test=turbo_test.iloc[:, -1]
  return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test=train_split(turbo1_train, turbo1_test)
y_test

20631    142
20632    141
20633    140
20634    139
20635    138
        ... 
33722     24
33723     23
33724     22
33725     21
33726     20
Name: RUL, Length: 13096, dtype: int64

In [ ]:
X2_train, y2_train, X2_test, y2_test=train_split(turbo2_train, turbo2_test)
X3_train, y3_train, X3_test, y3_test=train_split(turbo3_train, turbo3_test)
X4_train, y4_train, X4_test, y4_test=train_split(turbo4_train, turbo4_test)

In [ ]:
#separate the data into sequences of length 25 for each of the engines
sequence_length = 25
def gen_sequences_and_labels(df, seq_length):
    sequences = []
    labels = []
    for _, group in df.groupby('ENr'):
        if len(group) >= seq_length:
            for i in range(len(group) - seq_length + 1):
                sequence = group.iloc[i:i+seq_length, :-1].values
                sequences.append(sequence)
                rul_values = group.iloc[i+seq_length-1, -1]
                labels.append(rul_values)
    return np.array(sequences), np.array(labels)

train_sequences, train_labels = gen_sequences_and_labels(turbo1_test, sequence_length)
test_sequences, test_labels = gen_sequences_and_labels(turbo1_test, sequence_length)

print("Train Sequences shape:", train_sequences.shape)
print("Train Labels shape:", train_labels.shape)
print("Test Sequences shape:", test_sequences.shape)
print("Test Labels shape:", test_labels.shape)

Train Sequences shape: (10696, 25, 26)
Train Labels shape: (10696,)
Test Sequences shape: (10696, 25, 26)
Test Labels shape: (10696,)


In [ ]:
train_sequences2, train_labels2 = gen_sequences_and_labels(turbo2_test, sequence_length)
test_sequences2, test_labels2 = gen_sequences_and_labels(turbo2_test, sequence_length)

train_sequences3, train_labels3 = gen_sequences_and_labels(turbo3_test, sequence_length)
test_sequences3, test_labels3 = gen_sequences_and_labels(turbo3_test, sequence_length)

train_sequences4, train_labels4 = gen_sequences_and_labels(turbo4_test, sequence_length)
test_sequences4, test_labels4 = gen_sequences_and_labels(turbo4_test, sequence_length)

#Neural Networks construction

In [ ]:
#testing the asymmetric scoring function
def PHME_score(y_true, y_pred):
    dif=y_pred-y_true
    if dif>=0:
      score= math.exp(dif /10)-1
    else:
      score= math.exp(-dif /13)-1
    return score

In [ ]:
print("Score for an overestimation: ", PHME_score(112, 192))
print("Score for an underestimation: ", PHME_score(192, 112))

Score for an overestimation:  2979.9579870417283
Score for an underestimation:  469.5236172558757


In [ ]:
#metrics for the neural networks
def RMSE(y_true, y_pred):
    squared_errors = tf.square(y_true - y_pred)
    individual_rmse = tf.sqrt(tf.reduce_mean(squared_errors, axis=-1))
    return tf.keras.backend.mean(individual_rmse)

def PHME_score(y_true, y_pred):
    dif = y_pred - y_true
    positive_score = tf.exp(-dif / 13) - 1
    negative_score = tf.exp(-dif / 10) - 1
    individual_score = tf.where(dif >= 0, positive_score, negative_score)
    return tf.keras.backend.mean(individual_score)

def MAE(y_true, y_pred):
    absolute_errors = tf.abs(y_true - y_pred)
    individual_mae = tf.reduce_mean(absolute_errors, axis=-1)
    return tf.keras.backend.mean(individual_mae)

##FD001

In [ ]:
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', padding='same' , input_shape=(sequence_length, train_sequences.shape[2])))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.1))
model.add(Conv1D(32 , 3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=1))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=[RMSE, PHME_score, MAE])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 25, 64)            5056      
                                                                 
 max_pooling1d (MaxPooling1  (None, 25, 64)            0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 25, 64)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 25, 32)            6176      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 25, 32)            0         
 g1D)                                                            
                                                                 
 lstm (LSTM)                 (None, 25, 100)           5

In [ ]:
history = model.fit(train_sequences, train_labels, batch_size=10, epochs=30, validation_split=0.2, verbose=1)

Epoch 1/30
856/856 [==============================] - 18s 11ms/step - loss: 12756.4766 - RMSE: 99.1817 - PHME_score: 6949899264.0000 - MAE: 99.1817 - val_loss: 10082.5264 - val_RMSE: 83.9093 - val_PHME_score: 552624576.0000 - val_MAE: 83.9093
Epoch 2/30
856/856 [==============================] - 10s 11ms/step - loss: 6309.3994 - RMSE: 65.1676 - PHME_score: 121378512.0000 - MAE: 65.1676 - val_loss: 5190.1904 - val_RMSE: 57.0407 - val_PHME_score: 9813750.0000 - val_MAE: 57.0407
Epoch 3/30
856/856 [==============================] - 9s 10ms/step - loss: 3467.9949 - RMSE: 46.7365 - PHME_score: 3077831.5000 - MAE: 46.7365 - val_loss: 3675.9587 - val_RMSE: 47.7579 - val_PHME_score: 529621.6250 - val_MAE: 47.7579
Epoch 4/30
856/856 [==============================] - 8s 9ms/step - loss: 2904.6677 - RMSE: 42.6345 - PHME_score: 351995.6875 - MAE: 42.6345 - val_loss: 3069.5771 - val_RMSE: 40.9552 - val_PHME_score: 208935.7812 - val_MAE: 40.9552
Epoch 5/30
856/856 [==============================] -

In [ ]:
scores = model.evaluate(test_sequences, test_labels, verbose=1, batch_size=200)
print('\nRMSE: {}'.format(scores[1]))
print('\nPHME: {}'.format(scores[2]))
print('\nMAE: {}'.format(scores[3]))

y_pred = model.predict(test_sequences,verbose=1, batch_size=200)
y_true = y_test

54/54 [==============================] - 1s 7ms/step - loss: 776.5310 - RMSE: 18.5297 - PHME_score: 416.1861 - MAE: 18.5297

RMSE: 18.529733657836914

PHME: 416.1861267089844

MAE: 18.529733657836914
54/54 [==============================] - 1s 4ms/step


In [ ]:
predictions = model.predict(test_sequences)

r_squared = r2_score(train_labels, predictions)
print("R-squared:", r_squared)

335/335 [==============================] - 1s 3ms/step
R-squared: 0.7456530069118972


##FD002

In [ ]:
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', padding='same' ,  input_shape=(sequence_length, train_sequences2.shape[2])))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.1))
model.add(Conv1D(32 , 3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=1))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=[RMSE, PHME_score, MAE])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 25, 64)            5056      
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 25, 64)            0         
 g1D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 25, 64)            0         
                                                                 
 conv1d_3 (Conv1D)           (None, 25, 32)            6176      
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 25, 32)            0         
 g1D)                                                            
                                                                 
 lstm_2 (LSTM)               (None, 25, 100)          

In [ ]:
history = model.fit(train_sequences2, train_labels2, batch_size=10, epochs=30, validation_split=0.2, verbose=1)

Epoch 1/30
2223/2223 [==============================] - 26s 10ms/step - loss: 9032.6123 - RMSE: 76.9554 - PHME_score: 46625079296.0000 - MAE: 76.9554 - val_loss: 3660.5125 - val_RMSE: 49.3092 - val_PHME_score: 81640.9688 - val_MAE: 49.3092
Epoch 2/30
2223/2223 [==============================] - 22s 10ms/step - loss: 3827.2307 - RMSE: 49.6350 - PHME_score: 6406697.0000 - MAE: 49.6350 - val_loss: 3265.6301 - val_RMSE: 46.7838 - val_PHME_score: 13402.4893 - val_MAE: 46.7838
Epoch 3/30
2223/2223 [==============================] - 22s 10ms/step - loss: 3056.3945 - RMSE: 44.0125 - PHME_score: 2148247.2500 - MAE: 44.0125 - val_loss: 1892.9105 - val_RMSE: 36.2960 - val_PHME_score: 634.1173 - val_MAE: 36.2960
Epoch 4/30
2223/2223 [==============================] - 22s 10ms/step - loss: 2711.0334 - RMSE: 41.7882 - PHME_score: 463142.3750 - MAE: 41.7882 - val_loss: 1950.6652 - val_RMSE: 36.5627 - val_PHME_score: 297.3279 - val_MAE: 36.5627
Epoch 5/30
2223/2223 [==============================] - 2

In [ ]:
scores = model.evaluate(test_sequences2, test_labels2, verbose=1, batch_size=200)
print('\nRMSE: {}'.format(scores[1]))
print('\nPHME: {}'.format(scores[2]))
print('\nMAE: {}'.format(scores[3]))

y_pred = model.predict(test_sequences2,verbose=1, batch_size=200)
y_true = y_test

139/139 [==============================] - 1s 7ms/step - loss: 1300.9779 - RMSE: 28.3633 - PHME_score: 2082.6497 - MAE: 28.3633

RMSE: 28.363277435302734

PHME: 2082.649658203125

MAE: 28.363277435302734
139/139 [==============================] - 2s 5ms/step


In [ ]:
predictions = model.predict(test_sequences2)

r_squared = r2_score(train_labels2, predictions)
print("R-squared:", r_squared)

869/869 [==============================] - 3s 3ms/step
R-squared: 0.6400276919824353


##FD003

In [ ]:
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', padding='same' ,  input_shape=(sequence_length, train_sequences3.shape[2])))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.1))
model.add(Conv1D(32 , 3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=1))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=[RMSE, PHME_score, MAE])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 25, 64)            5056      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 25, 64)            0         
 g1D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 25, 64)            0         
                                                                 
 conv1d_5 (Conv1D)           (None, 25, 32)            6176      
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 25, 32)            0         
 g1D)                                                            
                                                                 
 lstm_4 (LSTM)               (None, 25, 100)          

In [ ]:
history = model.fit(train_sequences3, train_labels3, batch_size=10, epochs=30, validation_split=0.2, verbose=1)

Epoch 1/30
1136/1136 [==============================] - 16s 11ms/step - loss: 21697.3613 - RMSE: 123.3766 - PHME_score: 9658262609723392.0000 - MAE: 123.3766 - val_loss: 15970.4238 - val_RMSE: 99.4956 - val_PHME_score: 124933862588416.0000 - val_MAE: 99.4956
Epoch 2/30
1136/1136 [==============================] - 11s 10ms/step - loss: 11487.0000 - RMSE: 80.4909 - PHME_score: 70330567622656.0000 - MAE: 80.4909 - val_loss: 8950.9512 - val_RMSE: 71.1720 - val_PHME_score: 732836659200.0000 - val_MAE: 71.1720
Epoch 3/30
1136/1136 [==============================] - 13s 11ms/step - loss: 7269.9492 - RMSE: 62.2324 - PHME_score: 502606626816.0000 - MAE: 62.2324 - val_loss: 7220.5142 - val_RMSE: 65.7689 - val_PHME_score: 26000986112.0000 - val_MAE: 65.7689
Epoch 4/30
1136/1136 [==============================] - 12s 11ms/step - loss: 6101.6201 - RMSE: 56.8778 - PHME_score: 64275959808.0000 - MAE: 56.8778 - val_loss: 5886.5737 - val_RMSE: 54.0998 - val_PHME_score: 2748757760.0000 - val_MAE: 54.099

In [ ]:
scores = model.evaluate(test_sequences3, test_labels3, verbose=1, batch_size=200)
print('\nRMSE: {}'.format(scores[1]))
print('\nPHME: {}'.format(scores[2]))
print('\nMAE: {}'.format(scores[3]))

y_pred = model.predict(test_sequences3,verbose=1, batch_size=200)
y_true = y_test

71/71 [==============================] - 1s 6ms/step - loss: 2936.1201 - RMSE: 35.3571 - PHME_score: 492813536.0000 - MAE: 35.3571

RMSE: 35.35710906982422

PHME: 492813536.0

MAE: 35.35710906982422
71/71 [==============================] - 1s 3ms/step


In [ ]:
predictions = model.predict(test_sequences3)

r_squared = r2_score(train_labels3, predictions)
print("R-squared:", r_squared)

444/444 [==============================] - 1s 3ms/step
R-squared: 0.5572649407167675


##FD004

In [ ]:
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', padding='same' ,  input_shape=(sequence_length, train_sequences4.shape[2])))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.1))
model.add(Conv1D(32 , 3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=1))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=[RMSE, PHME_score, MAE])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 25, 64)            5056      
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 25, 64)            0         
 g1D)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 25, 64)            0         
                                                                 
 conv1d_7 (Conv1D)           (None, 25, 32)            6176      
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 25, 32)            0         
 g1D)                                                            
                                                                 
 lstm_6 (LSTM)               (None, 25, 100)          

In [ ]:
history = model.fit(train_sequences4, train_labels4, batch_size=10, epochs=30, validation_split=0.2, verbose=1)

Epoch 1/30
2823/2823 [==============================] - 35s 11ms/step - loss: 19239.5176 - RMSE: 109.2203 - PHME_score: 34012331348328448.0000 - MAE: 109.2203 - val_loss: 7858.5386 - val_RMSE: 65.8170 - val_PHME_score: 125825991049216.0000 - val_MAE: 65.8170
Epoch 2/30
2823/2823 [==============================] - 27s 10ms/step - loss: 8376.5234 - RMSE: 71.1472 - PHME_score: 334995947520.0000 - MAE: 71.1472 - val_loss: 7026.6357 - val_RMSE: 65.6278 - val_PHME_score: 1178945454080.0000 - val_MAE: 65.6278
Epoch 3/30
2823/2823 [==============================] - 27s 10ms/step - loss: 7816.7803 - RMSE: 69.4877 - PHME_score: 37551828992.0000 - MAE: 69.4877 - val_loss: 6836.4302 - val_RMSE: 64.6084 - val_PHME_score: 813439844352.0000 - val_MAE: 64.6084
Epoch 4/30
2823/2823 [==============================] - 27s 9ms/step - loss: 7718.7656 - RMSE: 69.3159 - PHME_score: 41799331840.0000 - MAE: 69.3159 - val_loss: 6255.7651 - val_RMSE: 60.8766 - val_PHME_score: 195800645632.0000 - val_MAE: 60.8766

In [ ]:
scores = model.evaluate(test_sequences4, test_labels4, verbose=1, batch_size=200)
print('\nRMSE: {}'.format(scores[1]))
print('\nPHME: {}'.format(scores[2]))
print('\nMAE: {}'.format(scores[3]))

y_pred = model.predict(test_sequences4,verbose=1, batch_size=200)
y_true = y_test

177/177 [==============================] - 1s 6ms/step - loss: 5608.3340 - RMSE: 55.1465 - PHME_score: 51553169408.0000 - MAE: 55.1465

RMSE: 55.146461486816406

PHME: 51553169408.0

MAE: 55.146461486816406
177/177 [==============================] - 2s 6ms/step


In [ ]:
predictions = model.predict(test_sequences4)

r_squared = r2_score(train_labels4, predictions)
print("R-squared:", r_squared)

1103/1103 [==============================] - 4s 3ms/step
R-squared: 0.3013967727039807
